# EWT

In [11]:
%run utils.ipynb

In [12]:
df = load_data('df_19_24_cleaned')

In [13]:
df = df['2022': '2024']

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24841 entries, 2022-01-01 00:00:00 to 2024-11-01 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          24841 non-null  float64
 1   solar          24841 non-null  int64  
 2   wind_onshore   24841 non-null  int64  
 3   wind_offshore  24841 non-null  int64  
 4   total_load     24841 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 1.1 MB


## Data Processing

### EWT

In [19]:
import pywt
# Define the features to decompose
features_to_decompose = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']

# Function to apply wavelet decomposition to each feature
def wavelet_decompose(series, wavelet='db1', level=3):
    coeffs = pywt.wavedec(series, wavelet, level=level)  # Decompose signal into frequency components
    decomposed_data = pd.DataFrame(coeffs).T  # Convert to DataFrame
    decomposed_data.columns = [f'{series.name}_wavelet_{i}' for i in range(len(coeffs))]
    return decomposed_data

# Apply wavelet decomposition to each feature and store the decomposed components
decomposed_features = []
for feature in features_to_decompose:
    decomposed = wavelet_decompose(df[feature], wavelet='db1', level=3)
    decomposed.index = df.index[:len(decomposed)]  # Align indices with the original data
    decomposed_features.append(decomposed)

# Concatenate decomposed features with the original data
decomposed_data = pd.concat(decomposed_features, axis=1)
data_dc = pd.concat([df, decomposed_data], axis=1)

In [20]:
data_dc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24841 entries, 2022-01-01 00:00:00 to 2024-11-01 00:00:00
Freq: h
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   price                    24841 non-null  float64
 1   solar                    24841 non-null  int64  
 2   wind_onshore             24841 non-null  int64  
 3   wind_offshore            24841 non-null  int64  
 4   total_load               24841 non-null  int64  
 5   price_wavelet_0          3106 non-null   float64
 6   price_wavelet_1          3106 non-null   float64
 7   price_wavelet_2          6211 non-null   float64
 8   price_wavelet_3          12421 non-null  float64
 9   wind_offshore_wavelet_0  3106 non-null   float64
 10  wind_offshore_wavelet_1  3106 non-null   float64
 11  wind_offshore_wavelet_2  6211 non-null   float64
 12  wind_offshore_wavelet_3  12421 non-null  float64
 13  wind_onshore_wavelet_0   3106 non